In [2]:
import json
import jsonlines
import random
import string

# 定义从文件中读取 JSON 数据的函数
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# 定义将结果写入文件的函数
def write_json_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        
# 假设输入文件路径和输出文件路径
input_file = './data/input_file_demo.json'  # 本地 JSON 文件路径
output_file = './data/out.json'  # 输出文件路径
# /home/lpc/repos/HotPotQA/ChatGLM_PEFT/main/data/train.json

In [3]:
data_list= read_json_file(input_file)

for data in data_list:
    if 'type' in data:
        del data['type']    
    if 'level' in data:
        del data['level']  

    #
    # 处理sp中的异常index
    #   
   
    valid_supporting_facts = []
    context = data["context"]
    # print(context[0][0])
    context = {item[0]: item[1] for item in context}
    # sp[0] :title 在context中是唯一标识符
    for sp in data["supporting_facts"]:
        if(sp[1]<len(context[sp[0]])):
            # print(sp[0])
            # print(context[sp[0]])
            valid_supporting_facts.append(sp)
    # print(len(valid_supporting_facts))
    sp_map = []
    for sp in valid_supporting_facts:
        if sp[0] not in sp_map:
            sp_map.append(sp[0])
    data["supporting_facts"] = valid_supporting_facts


    #
    # 剪除过多的context段落
    #  

    lenth = len(sp_map)
    # 选取有效和无效 1:1 所以只有<5的context才做剪除
    if lenth < 5:
        need_num = 0
        context = []
        for item in data["context"]:
            if item[0] in sp_map:
                context.append(item)
            elif need_num < lenth:
                context.append(item)
                need_num += 1
        random.shuffle(context)
        data["context"] = context
        # print(data["context"])
print(data_list)
    #
    # remove(item) bug
    #  （弃用）

    # print(data["context"])
        #     print(item)
        #     if item[0] not in sp_map and remove_num >= 0:
        #         # print(item[0])
        #         data["context"].remove(item)
        #         # print("context_len:"+ str(len(data["context"]))+"\n")
        #         remove_num -= 1
        #         # print("remove_num:"+ str(remove_num)+"\n")
        # print("\n")
        # random.shuffle(data["context"])
        # for item in data["context"]:
        #     print(item[0])




[{'_id': '5a8b57f25542995d1e6f1371', 'answer': 'yes', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]], 'context': [['Scott Derrickson', ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.', ' He lives in Los Angeles, California.', ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."']], ['Woodson, Arkansas', ['Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States.', ' Its population was 403 at the 2010 census.', ' It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area.', ' Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and businessman at the turn of the 20th century

In [4]:
#
#  把sp中的index换成原句，LLM对数字理解能力不强，难以定位到index
#  （为进一步压缩长度，并且提高预测结果，该方案已弃用）

# for data in data_list:
#     context = data["context"]
#     context = {item[0]: item[1] for item in context}
#     for sp in data["supporting_facts"]:
#         if(sp[1]<len(context[sp[0]])):
#             sp[1] = context[sp[0]][sp[1]]

# print(data_list[0]["context"]["Ed Wood (film)"][1])
# context = data[0]["context"]
# result = {item[0]: item[1] for item in context}
# print(result["Ed Wood (film)"][1])
# print(data_list[0]["supporting_facts"])

In [5]:
#
# 把context中的句子标上ABCD......XYZ
#

#
# 评估长度
#

# count = 0
# _sum = 0
# _max = -1
# out_26_count = 0
# out_52_count = 0
# for data in data_list:
#     sub = 0
#     for item in data['context']:
#         sub += len(item[1])
#     if sub > _max :
#         _max = sub
#     if sub > 26 :
#         out_26_count += 1
#     if sub > 52 :
#         out_52_count += 1
#     _sum += sub
#     count += 1

# print(f"总context句子数:{count}")
# print(f"每条数据平均contxet句子数:{_sum/count}")
# print(f"一条数据最多context句子数:{_max}")
# print(f"超出26条的条数:{out_26_count}")
# print(f"超出26条的占比:{out_26_count/count}")
# print(f"超出52条的条数:{out_52_count}")
# print(f"超出52条的占比:{out_52_count/count}")
        

In [6]:
#
# 把context中的句子标上ABCD......XYZ
# 
#print(data_list)

LETTERS = list(string.ascii_uppercase)#字母表
# print(LETTERS[0])
for data in data_list:
    context = data['context']
    new_context = []
    count = 0
    for item in context:
        paragragh = item[1]
        new_para = []
        for sent in paragragh:
            letter_index = count % 25
            letter = LETTERS[letter_index] 
            letter_suffix = int(count/26)
            key = letter + str(letter_suffix)
            new_sent_dict = {key:sent}
            new_para.append(new_sent_dict)
            count += 1 
        item[1] = new_para       
        new_context.append(item)
    data['context'] = new_context
    print(data['context'])      
        # print(item[1])


[['Scott Derrickson', [{'A0': 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.'}, {'B0': ' He lives in Los Angeles, California.'}, {'C0': ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."'}]], ['Woodson, Arkansas', [{'D0': 'Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States.'}, {'E0': ' Its population was 403 at the 2010 census.'}, {'F0': ' It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area.'}, {'G0': ' Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and businessman at the turn of the 20th century.'}, {'H0': ' Woodson is adjacent to the Wood Plantation, the largest of the plantations own by Ed Wood Sr.'}]], ['Ed Wood (film)', [{'I0': 'Ed Woo

In [7]:
#
# 把sp中的index替换为对应的ABCD标号
# 
for data in data_list:
    sp_list = data['supporting_facts']
    context = {item[0]: item[1] for item in data['context']}
    # print(context)
    new_sp_list = []
    for sp in sp_list:
        title = sp[0]
        sent_index = sp[1]
        # print(list(context[title][sent_index].keys())[0])
        sp[1] = list(context[title][sent_index].keys())[0]
    print(data['supporting_facts'])

[['Scott Derrickson', 'A0'], ['Ed Wood', 'L0']]
[['Kiss and Tell (1945 film)', 'G0'], ['Shirley Temple', 'E0'], ['Shirley Temple', 'F0']]
[['The Hork-Bajir Chronicles', 'A0'], ['The Hork-Bajir Chronicles', 'B0'], ['The Hork-Bajir Chronicles', 'C0'], ['Animorphs', 'I0'], ['Animorphs', 'J0']]
[['Laleli Mosque', 'M0'], ['Esma Sultan Mansion', 'L0']]
[['Big Stone Gap (film)', 'I0'], ['Adriana Trigiani', 'E0']]
[['2014 S/S', 'D0'], ['Winner (band)', 'O0']]


In [40]:
#
# 拆解为两个任务
# 任务1：回答sp,只需要title和ABCD标号
# 任务2：回答answer
# 
output = []
for data in data_list:
    fix_object_1 = {
        "conversations" :[]
    }
    fix_object_2 = {
        "conversations" :[]
    }
    # 任务1a
    question_object_1 = {
        "role": "user",
        "content": {
            "指令：请你根据context回答question,回答必须简短明晰。输入如下:"
            "question": data["question"],
            "context": data["context"]
        }
    }
    question_object_1["content"] = json.dumps(question_object_1["content"], ensure_ascii=False)
    answer_object_1 = {
        "role": "assistant",
        "content": data["answer"] 
    }
    answer_object_1["content"] = json.dumps(answer_object_1["content"], ensure_ascii=False)
    # print(type(answer_object_1['content']))
    fix_object_1["conversations"].append(question_object_1)
    fix_object_1["conversations"].append(answer_object_1)
    output.append(fix_object_1)
    # 任务2
    question_object_2 = {
        "role": "user",
        "content": {
            "指令：请你根据question和answer在context中寻找支持答案的语句,其中context中有多个item，每个item是由一个tiele(item的唯一表示)和一个语句列表组成的，每条语句前有语句标号如A0,B1(标号作为语句的唯一标识符)。你的回答必须按照以下给出的json格式的数组进行返回[[title,语句标号],[title,语句标号],.....]注意如果由多个语句支持答案，你需要找出最相关的几句话，title和语句标号必须与你找的item和语句的标号完全一致，方便我定位。输入如下:"
            "question":data["question"],
            "answer":data["answer"],
            "context":data["context"]
        }
    }
    question_object_2["content"] = json.dumps(question_object_2["content"], ensure_ascii=False)
    answer_object_2 = {
        "role": "assistant",
        "content":data['supporting_facts']
    }
    answer_object_2["content"] = json.dumps(answer_object_2["content"], ensure_ascii=False)
    fix_object_2["conversations"].append(question_object_2)
    fix_object_2["conversations"].append(answer_object_2)
    output.append(fix_object_2)
print(output)
# output = json.dumps(output, ensure_ascii=False)

[{'conversations': [{'role': 'user', 'content': '{"指令：请你根据context回答question,回答必须简短明晰。输入如下:question": "Were Scott Derrickson and Ed Wood of the same nationality?", "context": [["Scott Derrickson", [{"A0": "Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer."}, {"B0": " He lives in Los Angeles, California."}, {"C0": " He is best known for directing horror films such as \\"Sinister\\", \\"The Exorcism of Emily Rose\\", and \\"Deliver Us From Evil\\", as well as the 2016 Marvel Cinematic Universe installment, \\"Doctor Strange.\\""}]], ["Woodson, Arkansas", [{"D0": "Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States."}, {"E0": " Its population was 403 at the 2010 census."}, {"F0": " It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area."}, {"G0": " Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and 

In [41]:
# print(output)
with jsonlines.open(output_file, 'w') as writer:    
    for item in output:
        print(item)
        writer.write(item)


{'conversations': [{'role': 'user', 'content': '{"指令：请你根据context回答question,回答必须简短明晰。输入如下:question": "Were Scott Derrickson and Ed Wood of the same nationality?", "context": [["Scott Derrickson", [{"A0": "Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer."}, {"B0": " He lives in Los Angeles, California."}, {"C0": " He is best known for directing horror films such as \\"Sinister\\", \\"The Exorcism of Emily Rose\\", and \\"Deliver Us From Evil\\", as well as the 2016 Marvel Cinematic Universe installment, \\"Doctor Strange.\\""}]], ["Woodson, Arkansas", [{"D0": "Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States."}, {"E0": " Its population was 403 at the 2010 census."}, {"F0": " It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area."}, {"G0": " Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and b

In [104]:
# 将结果写入到输出文件中
# output_data =  json.dumps(output_data, ensure_ascii=False)
# write_json_file(output_file, output_data)
# with jsonlines.open(output_file, 'w') as writer:
#     writer.write(output_data)


In [105]:
# 读取输入
# input = read_json_file("output.json")
# str = json.dumps(input, ensure_ascii=False)
# print(str)